In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle
import folium
from shapely.geometry import LineString

In [2]:
df_street_busyness=pd.read_parquet('street_zones_factor.parquet')

In [22]:
df_street_busyness.shape

(159226, 5)

In [3]:
df_street_busyness.head(20)

osmid      highway  \
u        v           key                            
30807314 4332491202  0     147089927         path   
         2057103273  0      46613689    bridleway   
         6579409036  0      46613689    bridleway   
         1203806263  0     147089927         path   
30978752 4257949514  0     426425334      footway   
         4257949517  0    1081028223      footway   
         4257949513  0    1081028223      footway   
         1492282482  0     301283869      footway   
42421728 7784826655  0     195743153    secondary   
         7784826657  0     195743153    secondary   
                     0     195743153    secondary   
         7784826658  0     420625573    secondary   
42421731 10171363561 0       5671485  residential   
                     0       5671485  residential   
         10171363560 0     195743186  residential   
         10171363559 0     420625565    secondary   
         10171363562 0     420625570    secondary   
42421737 10171363583 0     195743201    secondary   
         10171363584 0     420625562      primary   
         10171363585 0     420625563      primary   

                                                                   geometry  \
u        v           key                                                      
30807314 4332491202  0    LINESTRING (-73.963576 40.79072, -73.963678 40...   
         2057103273  0    LINESTRING (-73.963576 40.79072, -73.963602 40...   
         6579409036  0    LINESTRING (-73.963576 40.79072, -73.963544 40...   
         1203806263  0    LINESTRING (-73.963576 40.79072, -73.963526 40...   
30978752 4257949514  0    LINESTRING (-73.97436 40.774776, -73.974413 40...   
         4257949517  0    LINESTRING (-73.97436 40.774776, -73.974398 40...   
         4257949513  0    LINESTRING (-73.97436 40.774776, -73.974312 40...   
         1492282482  0    LINESTRING (-73.97436 40.774776, -73.974179 40...   
42421728 7784826655  0    LINESTRING (-73.960044 40.798048, -73.959962 4...   
         7784826657  0    LINESTRING (-73.960044 40.798048, -73.960112 4...   
                     0    LINESTRING (-73.960044 40.798048, -73.960112 4...   
         7784826658  0    LINESTRING (-73.960044 40.798048, -73.960166 4...   
42421731 10171363561 0    LINESTRING (-73.961474 40.798654, -73.961536 4...   
                     0    LINESTRING (-73.961474 40.798654, -73.961536 4...   
         10171363560 0    LINESTRING (-73.961474 40.798654, -73.961401 4...   
         10171363559 0    LINESTRING (-73.961474 40.798654, -73.961392 4...   
         10171363562 0    LINESTRING (-73.961474 40.798654, -73.961581 4...   
42421737 10171363583 0    LINESTRING (-73.962873 40.799244, -73.962744 4...   
         10171363584 0    LINESTRING (-73.962873 40.799244, -73.9628 40....   
         10171363585 0    LINESTRING (-73.962873 40.799244, -73.962942 4...   

                          Taxi_Zone_ID  highway_factor  
u        v           key                                
30807314 4332491202  0            43.0             0.6  
         2057103273  0            43.0             0.4  
         6579409036  0            43.0             0.4  
         1203806263  0            43.0             0.6  
30978752 4257949514  0            43.0             0.6  
         4257949517  0            43.0             0.6  
         4257949513  0            43.0             0.6  
         1492282482  0            43.0             0.6  
42421728 7784826655  0            24.0             1.0  
         7784826657  0           151.0             1.0  
                     0            24.0             1.0  
         7784826658  0            24.0             1.0  
42421731 10171363561 0           151.0             0.6  
                     0            24.0             0.6  
         10171363560 0            24.0             0.6  
         10171363559 0            24.0             1.0  
         10171363562 0            24.0             1.0  
42421737 10171363583 0            24.0    

## test if there's obvious difference in rows with same osmip

In [4]:
test_data = df_street_busyness[(df_street_busyness['osmid'] == '426425334') ]

In [5]:
pd.set_option('display.max_colwidth', None)

print(test_data)

                               osmid  highway  \
u          v          key                       
30978752   4257949514 0    426425334  footway   
3156429739 4257949506 0    426425334  footway   
           4257949514 0    426425334  footway   
4257949506 3156429739 0    426425334  footway   
4257949514 3156429739 0    426425334  footway   
           30978752   0    426425334  footway   

                                                                         geometry  \
u          v          key                                                           
30978752   4257949514 0    LINESTRING (-73.97436 40.774776, -73.974413 40.774743)   
3156429739 4257949506 0    LINESTRING (-73.97449 40.774682, -73.974569 40.774625)   
           4257949514 0    LINESTRING (-73.97449 40.774682, -73.974413 40.774743)   
4257949506 3156429739 0    LINESTRING (-73.974569 40.774625, -73.97449 40.774682)   
4257949514 3156429739 0    LINESTRING (-73.974413 40.774743, -73.97449 40.774682)   
           3

In [10]:
# set centre of map
map_center = ( 40.774776,-73.97436)

# set up map 
m = folium.Map(location=map_center, zoom_start=50)

# test line string
line_strings = [
    ((-73.97436, 40.774776), (-73.974413, 40.774743)),((-73.97449, 40.774682), (-73.974569, 40.774625)),
    ((-73.97449, 40.774682), (-73.974413, 40.774743)),
    ((-73.974569, 40.774625), (-73.97449, 40.774682)),
    ((-73.974413, 40.774743), (-73.97449, 40.774682)),    ((-73.974413, 40.774743), (-73.97436, 40.774776))
]

# loop every line string
for line_string in line_strings:
    # set up Shapely LineString object
    line = LineString(line_string)

    # extract geolocation of line string
    points = [(point[1], point[0]) for point in line.coords]

    # set Folium PolyLine object and show it on map 
    folium.PolyLine(points, color='red', weight=2).add_to(m)


m


no apparent difference, combine same rows

In [25]:
filtered_data = df_street_busyness.groupby(['u', 'v']).first().reset_index()
print(type(filtered_data))

<class 'pandas.core.frame.DataFrame'>


In [26]:
filtered_data.head(2)

,u,v,osmid,highway,geometry,Taxi_Zone_ID,highway_factor
0,30807314,1203806263,147089927,path,"LINESTRING (-73.963576 40.79072, -73.963526 40.790692, -73.963489 40.790675, -73.963447 40.790664, -73.963394 40.790655, -73.963322 40.790647, -73.96326 40.790646)",43.0,0.6
1,30807314,2057103273,46613689,bridleway,"LINESTRING (-73.963576 40.79072, -73.963602 40.790704, -73.963632 40.790683, -73.963692 40.790636, -73.963773 40.790563, -73.963838 40.7905, -73.963888 40.790449, -73.96391 40.790417, -73.963927 40.79038, -73.963938 40.790317, -73.963929 40.79022, -73.963918 40.790129, -73.963902 40.789994, -73.963886 40.789897, -73.963863 40.78978, -73.963851 40.789708, -73.96384 40.789621, -73.963831 40.789538, -73.963828 40.789466, -73.963831 40.789389)",43.0,0.4


In [27]:
select_column=['osmid','highway','geometry','Taxi_Zone_ID','highway_factor']
filter_data=filtered_data[select_column]

In [28]:
filter_data.head(2)

,osmid,highway,geometry,Taxi_Zone_ID,highway_factor
0,147089927,path,"LINESTRING (-73.963576 40.79072, -73.963526 40.790692, -73.963489 40.790675, -73.963447 40.790664, -73.963394 40.790655, -73.963322 40.790647, -73.96326 40.790646)",43.0,0.6
1,46613689,bridleway,"LINESTRING (-73.963576 40.79072, -73.963602 40.790704, -73.963632 40.790683, -73.963692 40.790636, -73.963773 40.790563, -73.963838 40.7905, -73.963888 40.790449, -73.96391 40.790417, -73.963927 40.79038, -73.963938 40.790317, -73.963929 40.79022, -73.963918 40.790129, -73.963902 40.789994, -73.963886 40.789897, -73.963863 40.78978, -73.963851 40.789708, -73.96384 40.789621, -73.963831 40.789538, -73.963828 40.789466, -73.963831 40.789389)",43.0,0.4


In [33]:
filter_data.shape

(151858, 5)

In [32]:
filter_data['osmid'].duplicated().sum()

113945

In [39]:
duplicate_rows = filter_data[filter_data.duplicated(subset=['osmid', 'highway'], keep=False)]


duplicate_rows.head(10)

,osmid,highway,geometry,Taxi_Zone_ID,highway_factor
0,147089927,path,"LINESTRING (-73.963576 40.79072, -73.963526 40.790692, -73.963489 40.790675, -73.963447 40.790664, -73.963394 40.790655, -73.963322 40.790647, -73.96326 40.790646)",43.0,0.6
1,46613689,bridleway,"LINESTRING (-73.963576 40.79072, -73.963602 40.790704, -73.963632 40.790683, -73.963692 40.790636, -73.963773 40.790563, -73.963838 40.7905, -73.963888 40.790449, -73.96391 40.790417, -73.963927 40.79038, -73.963938 40.790317, -73.963929 40.79022, -73.963918 40.790129, -73.963902 40.789994, -73.963886 40.789897, -73.963863 40.78978, -73.963851 40.789708, -73.96384 40.789621, -73.963831 40.789538, -73.963828 40.789466, -73.963831 40.789389)",43.0,0.4
2,147089927,path,"LINESTRING (-73.963576 40.79072, -73.963678 40.790805)",43.0,0.6
3,46613689,bridleway,"LINESTRING (-73.963576 40.79072, -73.963544 40.79074, -73.963492 40.790768, -73.963444 40.790791, -73.963309 40.790858, -73.963201 40.790914, -73.963025 40.791033)",43.0,0.4
4,301283869,footway,"LINESTRING (-73.97436 40.774776, -73.974179 40.774907, -73.973964 40.775115, -73.973834 40.775257)",43.0,0.6
5,1081028223,footway,"LINESTRING (-73.97436 40.774776, -73.974312 40.77473)",43.0,0.6
6,426425334,footway,"LINESTRING (-73.97436 40.774776, -73.974413 40.774743)",43.0,0.6
7,1081028223,footway,"LINESTRING (-73.97436 40.774776, -73.974398 40.774809)",43.0,0.6
8,195743153,secondary,"LINESTRING (-73.960044 40.798048, -73.959962 40.798159)",24.0,1.0
9,195743153,secondary,"LINESTRING (-73.960044 40.798048, -73.960112 40.797955)",151.0,1.0


In [40]:
duplicate_rows.shape

(150168, 5)

In [43]:
unique_rows = filter_data.drop_duplicates(subset=['osmid', 'highway'], keep='first')
unique_rows.shape

(37914, 5)

In [44]:
unique_rows.head(10)

,osmid,highway,geometry,Taxi_Zone_ID,highway_factor
0,147089927,path,"LINESTRING (-73.963576 40.79072, -73.963526 40.790692, -73.963489 40.790675, -73.963447 40.790664, -73.963394 40.790655, -73.963322 40.790647, -73.96326 40.790646)",43.0,0.6
1,46613689,bridleway,"LINESTRING (-73.963576 40.79072, -73.963602 40.790704, -73.963632 40.790683, -73.963692 40.790636, -73.963773 40.790563, -73.963838 40.7905, -73.963888 40.790449, -73.96391 40.790417, -73.963927 40.79038, -73.963938 40.790317, -73.963929 40.79022, -73.963918 40.790129, -73.963902 40.789994, -73.963886 40.789897, -73.963863 40.78978, -73.963851 40.789708, -73.96384 40.789621, -73.963831 40.789538, -73.963828 40.789466, -73.963831 40.789389)",43.0,0.4
4,301283869,footway,"LINESTRING (-73.97436 40.774776, -73.974179 40.774907, -73.973964 40.775115, -73.973834 40.775257)",43.0,0.6
5,1081028223,footway,"LINESTRING (-73.97436 40.774776, -73.974312 40.77473)",43.0,0.6
6,426425334,footway,"LINESTRING (-73.97436 40.774776, -73.974413 40.774743)",43.0,0.6
8,195743153,secondary,"LINESTRING (-73.960044 40.798048, -73.959962 40.798159)",24.0,1.0
10,420625573,secondary,"LINESTRING (-73.960044 40.798048, -73.960166 40.798099)",24.0,1.0
11,420625565,secondary,"LINESTRING (-73.961474 40.798654, -73.961392 40.798618)",24.0,1.0
12,195743186,residential,"LINESTRING (-73.961474 40.798654, -73.961401 40.798754)",24.0,0.6
13,5671485,residential,"LINESTRING (-73.961474 40.798654, -73.961536 40.79857)",151.0,0.6


In [45]:
unique_rows.to_csv('predict_test.csv')

In [46]:
unique_rows['osmid'].duplicated().sum()

1